In [1]:
import os
from decouple import config
from openai import OpenAI

OPENAI_API_KEY = config("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
UPSTASH_VECTOR_REST_URL = config("UPSTASH_VECTOR_REST_URL")
UPSTASH_VECTOR_REST_TOKEN = config("UPSTASH_VECTOR_REST_TOKEN")

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import UpstashVectorStore

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
store = UpstashVectorStore(
    embedding=embeddings,
    index_url=UPSTASH_VECTOR_REST_URL,
    index_token=UPSTASH_VECTOR_REST_TOKEN
)

In [3]:
retriever = store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [7]:
# retriever.invoke("What is the city named after trees")

In [8]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

In [13]:
LLM_CONFIG = {
    "model": "gpt-4o-mini",
    "api_key": OPENAI_API_KEY
}

In [14]:
llm = ChatOpenAI(**LLM_CONFIG)

In [19]:
from langchain_core.prompts import ChatPromptTemplate

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

In [20]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] * 123121,
)

runnable.invoke({"num": 31})

{'passed': {'num': 31}, 'modified': 3816751}

In [21]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [24]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | parser

In [29]:
chain.invoke("What type of food is best known in Austin Texas")

'The provided context does not contain any information about the type of food best known in Austin, Texas.'